# InfoGAN

Ref.:
CHEN, Xi et al.  
Infogan: Interpretable representation learning by information maximizing generative adversarial nets.  
In: Advances in neural information processing systems. 2016. p. 2172-2180.

![InfoGAN architecture](arch.png)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

Using TensorFlow backend.


In [3]:
# Generator

In [4]:
def build_generator(channels, latent_dim):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation("tanh"))

    gen_input = Input(shape=(latent_dim,))
    img = model(gen_input)

    model.summary()

    return Model(gen_input, img)

## Discriminator

In [5]:
def build_disk_and_q_net(img_shape, num_classes):

    img = Input(shape=img_shape)

    # Shared layers between discriminator and recognition network
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Flatten())

    img_embedding = model(img)

    # Discriminator
    validity = Dense(1, activation='sigmoid')(img_embedding)

    # Recognition
    q_net = Dense(128, activation='relu')(img_embedding)
    label = Dense(num_classes, activation='softmax')(q_net)

    # Return discriminator and recognition network
    return Model(img, validity), Model(img, label)

## save D, G

In [6]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [7]:
def save_model(G, D):
    save(G, "generator")
    save(D, "discriminator")

## Train

In [8]:
def mutual_info_loss(c, c_given_x):
    """The mutual information metric we aim to minimize"""
    eps = 1e-8
    conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
    entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

    return conditional_entropy + entropy

In [9]:
def sample_generator_input(num_classes, batch_size):
    # Generator inputs
    sampled_noise = np.random.normal(0, 1, (batch_size, 62))
    sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    sampled_labels = to_categorical(sampled_labels, num_classes=num_classes)

    return sampled_noise, sampled_labels

In [10]:
def sample_images(G, num_classes, epoch):
    r, c = 10, 10

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        sampled_noise, _ = sample_generator_input(num_classes, c)
        label = to_categorical(np.full(fill_value=i, shape=(r,1)), num_classes=num_classes)
        gen_input = np.concatenate((sampled_noise, label), axis=1)
        gen_imgs = G.predict(gen_input)
        gen_imgs = 0.5 * gen_imgs + 0.5
        for j in range(r):
            axs[j,i].imshow(gen_imgs[j,:,:,0], cmap='gray')
            axs[j,i].axis('off')
    fig.savefig("images/%d.png" % epoch)
    plt.close()

In [11]:
def train(D, G, combined,
          num_classes,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and categorical labels
        sampled_noise, sampled_labels = sample_generator_input(num_classes, batch_size)
        gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

        # Generate a half batch of new images
        gen_imgs = G.predict(gen_input)

        # Train on real and generated data
        d_loss_real = D.train_on_batch(imgs, valid)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)

        # Avg. loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator and Q-network
        # ---------------------

        g_loss = combined.train_on_batch(gen_input, [valid, sampled_labels])

        # Plot the progress
        print ("%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, num_classes, epoch)

## main()

In [12]:
# create dirs
for d in ['images', 'saved_model']:
    if not os.path.exists(d):
        os.makedirs(d)

In [13]:
img_rows = 28
img_cols = 28
channels = 1
num_classes = 10

img_shape = (img_rows, img_cols, channels)
latent_dim = 72

In [14]:
optimizer = Adam(0.0002, 0.5)
losses = ['binary_crossentropy', mutual_info_loss]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Build and the discriminator and recognition network
D, auxilliary = build_disk_and_q_net(img_shape, num_classes)

D.compile(loss=['binary_crossentropy'],
          optimizer=optimizer,
          metrics=['accuracy'])

# Build and compile the recognition network Q
auxilliary.compile(loss=[mutual_info_loss],
                   optimizer=optimizer,
                   metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
# Build the generator
G = build_generator(channels, latent_dim)

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
gen_input = Input(shape=(latent_dim,))
img = G(gen_input)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              457856    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 128)      

In [17]:
# For the combined model we will only train the generator
D.trainable = False

# The discriminator takes generated image as input and determines validity
valid = D(img)
# The recognition network produces the label
target_label = auxilliary(img)

# The combined model  (stacked generator and discriminator)
combined = Model(gen_input, [valid, target_label])
combined.compile(loss=losses, optimizer=optimizer)

In [18]:
# epochs=50000
epochs=5000

In [19]:
train(D, G, combined, num_classes,
      epochs=epochs, batch_size=128, sample_interval=50)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.35, acc.: 28.12%] [Q loss: 0.94] [G loss: 3.23]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.71, acc.: 64.45%] [Q loss: 0.95] [G loss: 2.69]
2 [D loss: 0.30, acc.: 85.94%] [Q loss: 0.74] [G loss: 2.79]
3 [D loss: 0.08, acc.: 99.22%] [Q loss: 0.26] [G loss: 2.71]
4 [D loss: 0.07, acc.: 98.83%] [Q loss: 0.07] [G loss: 2.70]
5 [D loss: 0.04, acc.: 100.00%] [Q loss: 0.03] [G loss: 2.63]
6 [D loss: 0.04, acc.: 99.22%] [Q loss: 0.01] [G loss: 2.66]
7 [D loss: 0.03, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.67]
8 [D loss: 0.02, acc.: 99.61%] [Q loss: 0.00] [G loss: 2.48]
9 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.51]
10 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.58]
11 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.40]
12 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.47]
13 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.42]
14 [D loss: 0.01, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.32]
15 [D loss: 0.02, acc.: 100.00%] [Q loss: 0.00] [G loss: 2.31]
16 [D loss: 0.03, acc.: 98.83%] [Q loss: 0.00] [G loss: 2.28]
17 [D lo

133 [D loss: 0.54, acc.: 66.80%] [Q loss: 1.58] [G loss: 0.15]
134 [D loss: 0.63, acc.: 62.11%] [Q loss: 1.49] [G loss: 0.07]
135 [D loss: 0.61, acc.: 61.33%] [Q loss: 1.61] [G loss: 0.18]
136 [D loss: 0.57, acc.: 66.41%] [Q loss: 1.72] [G loss: 0.14]
137 [D loss: 0.62, acc.: 65.62%] [Q loss: 1.59] [G loss: 0.31]
138 [D loss: 0.57, acc.: 66.41%] [Q loss: 1.63] [G loss: 0.14]
139 [D loss: 0.58, acc.: 64.45%] [Q loss: 1.66] [G loss: 0.23]
140 [D loss: 0.55, acc.: 69.92%] [Q loss: 1.62] [G loss: 0.11]
141 [D loss: 0.66, acc.: 62.50%] [Q loss: 1.51] [G loss: 0.20]
142 [D loss: 0.56, acc.: 65.62%] [Q loss: 1.65] [G loss: 0.15]
143 [D loss: 0.63, acc.: 62.11%] [Q loss: 1.58] [G loss: 0.18]
144 [D loss: 0.57, acc.: 64.84%] [Q loss: 1.63] [G loss: 0.13]
145 [D loss: 0.55, acc.: 68.36%] [Q loss: 1.66] [G loss: 0.10]
146 [D loss: 0.56, acc.: 66.02%] [Q loss: 1.51] [G loss: 0.11]
147 [D loss: 0.62, acc.: 64.45%] [Q loss: 1.54] [G loss: 0.15]
148 [D loss: 0.63, acc.: 65.23%] [Q loss: 1.69] [G loss

...

4403 [D loss: 0.70, acc.: 50.78%] [Q loss: 0.65] [G loss: 0.00]
4404 [D loss: 0.69, acc.: 50.78%] [Q loss: 0.64] [G loss: 0.00]
4405 [D loss: 0.70, acc.: 50.78%] [Q loss: 0.65] [G loss: 0.00]
4406 [D loss: 0.70, acc.: 51.17%] [Q loss: 0.65] [G loss: 0.04]
4407 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.64] [G loss: 0.00]
4408 [D loss: 0.69, acc.: 51.56%] [Q loss: 0.65] [G loss: 0.00]
4409 [D loss: 0.69, acc.: 51.17%] [Q loss: 0.66] [G loss: 0.00]
4410 [D loss: 0.69, acc.: 51.95%] [Q loss: 0.66] [G loss: 0.00]
4411 [D loss: 0.69, acc.: 53.52%] [Q loss: 0.66] [G loss: 0.02]
4412 [D loss: 0.69, acc.: 50.78%] [Q loss: 0.64] [G loss: 0.00]
4413 [D loss: 0.70, acc.: 51.17%] [Q loss: 0.65] [G loss: 0.01]
4414 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.65] [G loss: 0.00]
4415 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.65] [G loss: 0.00]
4416 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.64] [G loss: 0.00]
4417 [D loss: 0.69, acc.: 50.39%] [Q loss: 0.65] [G loss: 0.00]
4418 [D loss: 0.69, acc.: 51.56%] [Q los

4532 [D loss: 0.69, acc.: 54.69%] [Q loss: 0.67] [G loss: 0.00]
4533 [D loss: 0.70, acc.: 48.83%] [Q loss: 0.66] [G loss: 0.00]
4534 [D loss: 0.69, acc.: 50.39%] [Q loss: 0.67] [G loss: 0.00]
4535 [D loss: 0.70, acc.: 51.56%] [Q loss: 0.66] [G loss: 0.01]
4536 [D loss: 0.69, acc.: 52.34%] [Q loss: 0.65] [G loss: 0.00]
4537 [D loss: 0.68, acc.: 57.03%] [Q loss: 0.67] [G loss: 0.05]
4538 [D loss: 0.69, acc.: 51.95%] [Q loss: 0.66] [G loss: 0.00]
4539 [D loss: 0.70, acc.: 55.08%] [Q loss: 0.66] [G loss: 0.01]
4540 [D loss: 0.69, acc.: 55.47%] [Q loss: 0.66] [G loss: 0.00]
4541 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.66] [G loss: 0.00]
4542 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.68] [G loss: 0.00]
4543 [D loss: 0.70, acc.: 50.39%] [Q loss: 0.67] [G loss: 0.01]
4544 [D loss: 0.69, acc.: 52.73%] [Q loss: 0.67] [G loss: 0.00]
4545 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.67] [G loss: 0.01]
4546 [D loss: 0.70, acc.: 53.91%] [Q loss: 0.66] [G loss: 0.00]
4547 [D loss: 0.69, acc.: 53.52%] [Q los

4661 [D loss: 0.69, acc.: 53.91%] [Q loss: 0.67] [G loss: 0.00]
4662 [D loss: 0.69, acc.: 56.64%] [Q loss: 0.68] [G loss: 0.03]
4663 [D loss: 0.70, acc.: 51.56%] [Q loss: 0.65] [G loss: 0.02]
4664 [D loss: 0.70, acc.: 48.83%] [Q loss: 0.66] [G loss: 0.00]
4665 [D loss: 0.69, acc.: 50.39%] [Q loss: 0.66] [G loss: 0.05]
4666 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.65] [G loss: 0.00]
4667 [D loss: 0.69, acc.: 50.39%] [Q loss: 0.66] [G loss: 0.00]
4668 [D loss: 0.70, acc.: 50.00%] [Q loss: 0.65] [G loss: 0.03]
4669 [D loss: 0.69, acc.: 51.56%] [Q loss: 0.65] [G loss: 0.00]
4670 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.67] [G loss: 0.01]
4671 [D loss: 0.69, acc.: 52.34%] [Q loss: 0.67] [G loss: 0.05]
4672 [D loss: 0.70, acc.: 50.00%] [Q loss: 0.66] [G loss: 0.01]
4673 [D loss: 0.69, acc.: 53.12%] [Q loss: 0.66] [G loss: 0.00]
4674 [D loss: 0.69, acc.: 52.34%] [Q loss: 0.66] [G loss: 0.03]
4675 [D loss: 0.69, acc.: 53.91%] [Q loss: 0.66] [G loss: 0.01]
4676 [D loss: 0.69, acc.: 53.12%] [Q los

4790 [D loss: 0.70, acc.: 46.88%] [Q loss: 0.65] [G loss: 0.00]
4791 [D loss: 0.69, acc.: 54.30%] [Q loss: 0.66] [G loss: 0.00]
4792 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.65] [G loss: 0.01]
4793 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.65] [G loss: 0.00]
4794 [D loss: 0.69, acc.: 52.34%] [Q loss: 0.67] [G loss: 0.01]
4795 [D loss: 0.69, acc.: 53.91%] [Q loss: 0.66] [G loss: 0.00]
4796 [D loss: 0.69, acc.: 51.17%] [Q loss: 0.66] [G loss: 0.01]
4797 [D loss: 0.69, acc.: 51.95%] [Q loss: 0.67] [G loss: 0.00]
4798 [D loss: 0.69, acc.: 54.30%] [Q loss: 0.65] [G loss: 0.00]
4799 [D loss: 0.70, acc.: 50.78%] [Q loss: 0.66] [G loss: 0.01]
4800 [D loss: 0.69, acc.: 53.12%] [Q loss: 0.67] [G loss: 0.00]
4801 [D loss: 0.69, acc.: 51.17%] [Q loss: 0.67] [G loss: 0.00]
4802 [D loss: 0.69, acc.: 51.56%] [Q loss: 0.67] [G loss: 0.02]
4803 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.66] [G loss: 0.01]
4804 [D loss: 0.70, acc.: 51.95%] [Q loss: 0.66] [G loss: 0.00]
4805 [D loss: 0.69, acc.: 52.34%] [Q los

4919 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.68] [G loss: 0.00]
4920 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.67] [G loss: 0.00]
4921 [D loss: 0.70, acc.: 50.78%] [Q loss: 0.67] [G loss: 0.00]
4922 [D loss: 0.69, acc.: 50.39%] [Q loss: 0.67] [G loss: 0.00]
4923 [D loss: 0.70, acc.: 50.39%] [Q loss: 0.67] [G loss: 0.00]
4924 [D loss: 0.70, acc.: 48.83%] [Q loss: 0.67] [G loss: 0.01]
4925 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.67] [G loss: 0.00]
4926 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.68] [G loss: 0.00]
4927 [D loss: 0.70, acc.: 49.61%] [Q loss: 0.68] [G loss: 0.00]
4928 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.67] [G loss: 0.00]
4929 [D loss: 0.70, acc.: 52.34%] [Q loss: 0.68] [G loss: 0.09]
4930 [D loss: 0.70, acc.: 52.73%] [Q loss: 0.67] [G loss: 0.00]
4931 [D loss: 0.70, acc.: 49.22%] [Q loss: 0.68] [G loss: 0.00]
4932 [D loss: 0.70, acc.: 51.56%] [Q loss: 0.67] [G loss: 0.02]
4933 [D loss: 0.70, acc.: 47.66%] [Q loss: 0.67] [G loss: 0.01]
4934 [D loss: 0.70, acc.: 51.56%] [Q los